# Assistant-10<br/>Knowledge Retrieval

### Get an OpenAI client

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [5]:
import oaihelper as helper

## Get an OpenAI client
client = helper.get_openai_client(api_key=helper.api_KEY,
        api_version=helper.api_version,
        azure_endpoint=helper.api_URI,)

### Load the files

In [6]:
## Files
file_benefits = helper.upload_file(client,"../data/HR/contoso-benefits.txt")
file_401k = helper.upload_file(client,"../data/HR/contoso-401k.txt")
file_leave = helper.upload_file(client,"../data/HR/contoso-medical_leave.txt")

Added file:  assistant-qgRqLR622U1TP2WZsRFvDPxg 4
Added file:  assistant-oXCxCoXEbn0ja8FjLnXaD9R1 5
Added file:  assistant-n5HmS8oknuGEDtb1q2b3JK2B 6


### Define the tools to use in the Assistants

In [7]:
tools_list = [        
        {
            "type": "function",
            "function": {
                "name": "send_email",
                "description": "Sends an email to a recipient(s).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "to": {
                            "type": "string",
                            "description": "The email(s) the email should be sent to."
                        },
                        "content": {
                            "type": "string",
                            "description": "The content of the email."
                        }
                    },
                    "required": ["to", "content"]
                }
            }
        },
        {"type","retrieval"}
    ]

### Define the HR Assistant

In [8]:
helper.clear_shelves()

hr_assistant = helper.create_assistant(client,
        name="HR Assistant",
        instructions="You HR support chatbot. Use your knowledge base to best respond to the employee queries. Be friendly, professional, and helpful. If you don't know the answer, please say, 'contact HR for more information.'",
        tools=[{"type": "retrieval"}],
        model=helper.gpt_deployment_name,
        file_ids=[file_benefits.id, file_401k.id, file_leave.id],)

Added assistant:  asst_ueQ8Dm9CKiMz50BMQNBR875g 2


### Define the delegate to handle function calling

In [9]:
def function_calling_delegate(client, run, thread):
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json
    for action in required_actions["tool_calls"]:
        func_name = action['function']['name']
        arguments = json.loads(action['function']['arguments'])
        
        if func_name == "send_email":
            print("Sending email...")
            email_to = arguments['to']
            email_content = arguments['content']
            helper.send_email(email_to,email_content)
            
            tool_outputs.append({
                "tool_call_id": action['id'],
                "output": "Email sent"
            })
        else:
            raise ValueError(f"Unknown function: {func_name}")
        
    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

### Process user messages

In [10]:
try:
    helper.generate_response(client,hr_assistant,"Can you list some company benefits? Can you provide a summary for requesting medical leave?", "123", "Alex", function_calling_delegate)
    helper.generate_response(client,hr_assistant,"Can you provide me a complete list of company benefits benefits?", "123", "Alex", function_calling_delegate)
    helper.generate_response(client,hr_assistant,"What is the 401k matching percentage?", "123", "Alex", function_calling_delegate)
finally:
    helper.cleanup(client)

Creating new thread for Alex with user_id 123
Added thread:  thread_DwXVkwWcHCMI6tXA0YuHVERL 1
Generated message: Here are some of the company benefits available to you:

- **Health Insurance**: The company covers 100% of the premiums for medical, dental, and vision plans for employees and their dependents, including flexible spending accounts, health savings accounts, and wellness programs.
- **Retirement Savings**: Employees receive a 50% match on their 401(k) contributions, up to 6% of their salary. There are also financial education and planning services available.
- **Paid Time Off**: There are generous policies for vacation, sick leave, holidays, parental leave, and sabbatical leave.
- **Professional Development**: Opportunities for learning new skills, advancing careers, and pursuing passions are provided, including tuition reimbursement, online courses, mentoring programs, and internal mobility.
- **Work-Life Balance**: The company supports flexible work arrangements, remote wo